In [1]:
import re

_camel_re1 = re.compile('(.)([A-Z][a-z]+)')
_camel_re2 = re.compile('([a-z0-9])([A-Z])')
def camel2snake(name):
    s1 = re.sub(_camel_re1, r'\1_\2', name)
    return re.sub(_camel_re2, r'\1_\2', s1).lower()

class Callback():
    _order=0
    def set_runner(self, run): self.run=run
    def __getattr__(self, k):
        print("Getattr of parent callback called")
        return getattr(self.run, k)
    @property
    def name(self):
        name = re.sub(r'Callback$', '', self.__class__.__name__)
        return camel2snake(name or 'callback')

In [85]:
class TrainEvalCallback(Callback):
    def begin_fit(self):
        self.run.n_epochs=0.
        self.run.n_iter=0
    
    def after_batch(self):
        if not self.in_train: return
        self.run.n_epochs += 1./self.iters
        self.run.n_iter   += 1
        
    def begin_epoch(self):
        print(dir(self))
        self.run.n_epochs=0
        print(self.run,"Hello")
        print(self.a)
        print(self.model)
        self.run.in_train=True

    def begin_validate(self):
        self.model.eval()
        self.run.in_train=False

In [86]:
from typing import *

def listify(o):
    if o is None: return []
    if isinstance(o, list): return o
    if isinstance(o, Iterable): return list(o)
    return [o]

In [87]:
class Runner:
    """
    Contains the one_batch,all_batches and fit function
    """
    
    def __init__(self, cbs=None, cb_funcs=None):
        cbs = listify(cbs)
        
        self.stop,self.cbs = False,[TrainEvalCallback()]+cbs
    
    @property
    def model(self):     return "This is a model"
        
    def __call__(self, cb_name):
        for cb in sorted(self.cbs, key=lambda x: x._order):
            f = getattr(cb, cb_name, None)
            print("callable")
            print(f)
            if f and f(): return True
        return False
    
    def fit(self):
        print(self.cbs)
        for cb in self.cbs:
            cb.set_runner(self)
            self.a = "Abhinav"
            if not self("begin_epoch"):
                print(cb.n_epochs,"n_epochs")
                
            
        

In [88]:
run = Runner()

In [89]:

run.fit()

callable
<bound method TrainEvalCallback.begin_epoch of <__main__.TrainEvalCallback object at 0x10f2d17b8>>
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_order', 'after_batch', 'begin_epoch', 'begin_fit', 'begin_validate', 'name', 'run', 'set_runner']
<__main__.Runner object at 0x10f2d1940> Hello
Getattr of parent callback called
Abhinav
Getattr of parent callback called
This is a model
Getattr of parent callback called
0 n_epochs
